# LangChain
* a framework that is focused on building applications that implement LLMs
* we are going to use OpenAI models within LangChain and utilize some helpful tools that are a bit more tailored for the user experience

### First lets setup our API connection with OpenAI
* The only difference here, is that we are giving our API key to LangChain's OpenAI object (langchain is our middle man)

In [8]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

In [9]:
load_dotenv("C:\\Users\\Patrick\\Desktop\\PROJECTS\\Python Lesson Plan\\AI_CLASS_7\\api.env")  # could pass in the path of the .env file in the arguments
openai_api_key=os.environ.get('OPENAI_API_KEY')
# print(openai_api_key)
# We also call the model here
llm = ChatOpenAI(api_key=openai_api_key, model="gpt-3.5-turbo")

We can then use the .invoke() function to make an API call
* So much easier
* All that's needed is the prompt
* The response object looks very similar to OpenAI's

In [11]:
response = llm.invoke("Tell me a joke")
print(response.content)

Why did the scarecrow win an award?
Because he was outstanding in his field!


## Prompt Templates
* We can build out more detailed prompts by using prompt templates

In [12]:
# Using ChatPromptTemplate we can create a chat history that is simpler than OpenAI's format
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class comedian who focuses on AI."),
    ("user", "{input}")
])
chain = prompt | llm 
chain.invoke({"input": "Tell me a joke"})

AIMessage(content="Why did the AI break up with his internet girlfriend?\n\nBecause he couldn't handle her bandwidth!", response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 25, 'total_tokens': 44}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0a4c3257-54c0-49e2-9165-ed0cf38fc77a-0', usage_metadata={'input_tokens': 25, 'output_tokens': 19, 'total_tokens': 44})

In [15]:
# We can make it more task specific and user friendly
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class comedian."),
    # We create the variable 'topic' here that the user can fill in
    ("user", "Tell me a joke about {topic}")
])
chain = prompt | llm 
# This is where the user will fill in the variable 'topic'
chain.invoke({"topic": "AI"})

AIMessage(content='Why did the robot go on a diet?\n\nBecause it had too many bytes!', response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 23, 'total_tokens': 39}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-8bdf70de-4be0-4692-bc3c-42ee6c196e5d-0')

### Partial Prompt Templates
* Separates the core prompt from specific details 
    * keeps your code cleaner and more modular
* the real power lies in calling functions to handle calculations as partial variables

In [3]:
from datetime import datetime

# Using the 'datetime' library, we made a function that fetches the current date from the device
def get_datetime():
    now = datetime.now()
    return now.strftime("%m/%d/%Y")

# lets run it and test it out
get_datetime()

'07/13/2024'

In [16]:
# This looks like a normal prompt template but with the partial_variables parameter added and set date to the function to get the date
prompt = PromptTemplate(
    template="Tell me a {adjective} joke about the day {date}",
    input_variables=["adjective"],
    # This makes the prompt template essentially update the date everytime the model gets prompted
    partial_variables={"date": get_datetime},
)
# Create the chain with the prompt template and the model
chain = prompt | llm
# .invoke() to prompt the model and we add our adjective - this is what the user would put into this hypothetical AI application
response = chain.invoke({"adjective": "funny"})
# Display the output by just using .content
print(response.content)

Why was 07/13/2024 such a great day? Because it was the only day when everyone agreed that Mondays aren't so bad after all!


### Few-shot Prompt Template
* using the FewShotPromptTemplate object to incorporate examples for few shot prompt engineering
* examples and example_prompt parameters are required in the FewShotPromptTemplate
    * The object will pass the examples through the example_prompt 

In [1]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

examples = [
    {
        "question": "Who lived longer, Muhammad Ali or Alan Turing?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali
""",
    },
    {
        "question": "When was the founder of craigslist born?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952
""",
    },
    {
        "question": "Who was the maternal grandfather of George Washington?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball
""",
    },
    {
        "question": "Are both the directors of Jaws and Casino Royale from the same country?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who is the director of Jaws?
Intermediate Answer: The director of Jaws is Steven Spielberg.
Follow up: Where is Steven Spielberg from?
Intermediate Answer: The United States.
Follow up: Who is the director of Casino Royale?
Intermediate Answer: The director of Casino Royale is Martin Campbell.
Follow up: Where is Martin Campbell from?
Intermediate Answer: New Zealand.
So the final answer is: No
""",
    },
]

In [2]:
example_prompt = PromptTemplate(
    input_variables=["question", "answer"], template="Question: {question}\n{answer}"
)

print(example_prompt.format(**examples[0]))

Question: Who lived longer, Muhammad Ali or Alan Turing?

Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali



In [3]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)

print(prompt.format(input="Who was the father of Mary Ball Washington?"))

Question: Who lived longer, Muhammad Ali or Alan Turing?

Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali


Question: When was the founder of craigslist born?

Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952


Question: Who was the maternal grandfather of George Washington?

Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball W

## Output Parser
* Whenever we make an API call using the .invoke() function, we get an AIMessage response object returned back to us
* When building applications, all we need to display is the generated content of the response object. 
    * The object does hold some helpful information that can be used as well, but that is a case by case basis

In [18]:
# Here we use the StrOutputParser to get the output as a string
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
# We can just add it in the chain to be incorporated in the .invoke() function
# Uses the same prompt template as before (scroll up and find the prompt variable)
chain = prompt | llm | output_parser
# Notice how the output is now just the content as a string
chain.invoke({"topic": "math"})

"Why was the equal sign so humble?\n\nBecause he knew he wasn't less than or greater than anyone else!"

There are many other Output Parsers available in LangChain.
Accessible here: https://python.langchain.com/v0.1/docs/modules/model_io/output_parsers/ 

In [5]:
# We can also use the CommaSeparatedListOutputParser to get the output as a list
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate

output_parser = CommaSeparatedListOutputParser()
# We take the format instructions from the output_parser to feed into the prompt template. This allows the model to understand the format of the input when generating it
format_instructions = output_parser.get_format_instructions()

# We create a new prompt template that takes in the format instructions
# We use the partial variables to pass in the format instructions because the format instructions are not known until the output_parser is created
prompt = PromptTemplate(
    template="List five synonyms for {word}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

# We create the chain to incorporate the prompt, the model, and the output parser
chain = prompt | llm | output_parser
# We invoke the chain with the variable 'word' filled in
chain.invoke({"word": "happy"})

['joyful', 'delighted', 'content', 'cheerful', 'elated']

## Exercise

Chain together a prompt template, an OpenAI model, and an output parser together to make a recipe generator for any meal the user inputs.

## Prompt Chaining
* a technique that breaks down a prompt into multiple subtasks
* typically used for longer generations or complex tasks that need more direction in its execution

In [21]:
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain_core.prompts import PromptTemplate

# This is an LLMChain to write a synopsis given a title of a play.
template = """You are a playwright. Given the title of play, it is your job to write a synopsis for that title.

Title: {title}
Playwright: This is a synopsis for the above play:"""
prompt_template = PromptTemplate(input_variables=["title"], template=template)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template)

# This is an LLMChain to write a review of a play given a synopsis.
template = """You are a play critic from the New York Times. Given the synopsis of play, it is your job to write a review for that play.

Play Synopsis:
{synopsis}
Review from a New York Times play critic of the above play:"""
prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template)

# This is the overall chain where we run these two chains in sequence.
overall_chain = SimpleSequentialChain(
    chains=[synopsis_chain, review_chain], verbose=True
)

review = overall_chain.run("Tragedy at sunset on the beach")



> Entering new SimpleSequentialChain chain...
"Tragedy at Sunset on the Beach" follows the story of a group of friends who gather for a relaxing evening by the ocean. As the sun sets and tensions rise, long-buried secrets and conflicts come to light, leading to a tragic event that changes their lives forever. Set against the backdrop of a picturesque beach, this play explores themes of friendship, betrayal, and the consequences of our actions as the characters struggle to come to terms with the heartbreaking events that unfold before them.
"Tragedy at Sunset on the Beach" is a gripping and emotionally charged play that delves into the complexities of human relationships and the devastating consequences of unresolved conflicts. Set against the stunning backdrop of a beach at sunset, the play unfolds with a sense of impending doom as long-buried secrets come to light and tensions reach a boiling point.

The talented ensemble cast delivers powerful performances, capturing the raw emotio

## Tool Calling
* synonymous with the term function calling
* allows the model being used in the application to choose a function from the list provided instead of generating a natural language response
* define our own tools using the ‘@tool’ decorator
    * there has to be a docstring that describes the function in natural language for the model to understand the proper use of the function

In [37]:
from langchain.tools import tool

@tool # tool decorator
def add(x: int, y: int) -> int:
    '''Adds two numbers together.''' # Docstring 
    return x + y

@tool
def multiply(x: int, y: int) -> int:
    '''Multiplies two numbers together.'''
    return x * y

tools = [add, multiply]

llm_with_tools = llm.bind_tools(tools)

prompt = "What is 257 + 243? Also, what is 223 * 3?"
result = llm_with_tools.invoke(prompt)
print(result)
print(result.tool_calls)
# some models may also add content to explain the reasoning behind the function calls
print(result.content)

content='' additional_kwargs={'tool_calls': [{'id': 'call_M25FBFSr4vtmlORIFU4XmR0h', 'function': {'arguments': '{"x": 257, "y": 243}', 'name': 'add'}, 'type': 'function'}, {'id': 'call_5GjVtI1V5SWRZ23aJHNjtcAE', 'function': {'arguments': '{"x": 223, "y": 3}', 'name': 'multiply'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 112, 'total_tokens': 161}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-2fa80aa9-ff08-4975-b58f-3bf6d75a7159-0' tool_calls=[{'name': 'add', 'args': {'x': 257, 'y': 243}, 'id': 'call_M25FBFSr4vtmlORIFU4XmR0h'}, {'name': 'multiply', 'args': {'x': 223, 'y': 3}, 'id': 'call_5GjVtI1V5SWRZ23aJHNjtcAE'}]
[{'name': 'add', 'args': {'x': 257, 'y': 243}, 'id': 'call_M25FBFSr4vtmlORIFU4XmR0h'}, {'name': 'multiply', 'args': {'x': 223, 'y': 3}, 'id': 'call_5GjVtI1V5SWRZ23aJHNjtcAE'}]



Here are a list of available tools: https://python.langchain.com/v0.2/docs/integrations/tools/

Here are a list of toolkits that hold alike tools: https://python.langchain.com/v0.2/docs/integrations/toolkits/ 

## Exercise

Write a tool calling chain that converts USD to two different currencies (ex: the euro and Japanese yen).

## Agents
* An agent chooses the sequence of actions to take using a reasoning engine
* This allows the model to take control and complete more versatile tasks instead of being hard coded down a path in a chain

In [42]:
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor
from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are mathematical genius that can answer any math question."),
    ("user", "{input}"),
    MessagesPlaceholder("agent_scratchpad"),
])

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input": "What is 257 + 243? Also, what is that result mutliplied by 3?"})



> Entering new AgentExecutor chain...

Invoking: `add` with `{'x': 257, 'y': 243}`


500
Invoking: `multiply` with `{'x': 500, 'y': 3}`


1500The sum of 257 and 243 is 500. When this result is multiplied by 3, the answer is 1500.

> Finished chain.


{'input': 'What is 257 + 243? Also, what is that result mutliplied by 3?',
 'output': 'The sum of 257 and 243 is 500. When this result is multiplied by 3, the answer is 1500.'}

Here is more information on Agents: https://python.langchain.com/v0.1/docs/modules/agents/concepts/ 

Here is a list of Agent types available in LangChain: https://python.langchain.com/v0.1/docs/modules/agents/agent_types/ 

## Exercise:

Use one (or more) of the simple to use tools that LangChain provides to create an agent. (AlphaVantage, Brave Search, Dall-E Image Generator, OpenWeatherMap, SerpAPI, WolframAlpha, etc.) 